In [1]:
import numpy as np
import pandas as pd
from astropy.io import fits
from altaipony.flarelc import FlareLightCurve
import sys

import matplotlib.pyplot as plt
%matplotlib inline

from funcs.helper import write_flares_to_file, read_custom_aperture_lc
from funcs.custom_detrending import custom_detrending
import os

/usr/lib/python3.6/importlib/_bootstrap_external.py:426: ImportWarning: Not importing directory /home/ekaterina/Documents/001_Science/Flares_in_Clusters_II/flaresinclustersii/lib/python3.6/site-packages/mpl_toolkits: missing __init__
  _warnings.warn(msg.format(portions[0]), ImportWarning)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
CWD = "/".join(os.getcwd().split("/")[:-1])
paths = pd.read_csv(f"{CWD}/custom_aperture/lclist.txt", header=None)
paths = list(paths[0].values)
len(paths)

191

In [3]:
for i, path in enumerate(paths[:1]):
    print(path, "\n")
    path = f"{CWD}/custom_aperture/" + path
    cluster = f"{CWD}/flare_tables/2020_02_05_TESSUCDs"

    flc = read_custom_aperture_lc(path)

    flcd = flc.detrend("custom", func=custom_detrending)
    
    flcd = flcd.find_flares()
    
    write_flares_to_file(flcd, cluster)

    print("\nFinished TIC {} ({})\n------------------------------\n".format(flc.targetid, flc.campaign))
    flc.plot()
    plt.plot(flcd.time, flcd.detrended_flux+50)
    plt.ylim(500,700)
    
    max_ampl = 2*flcd.flares.ampl_rec.max()
    #print(max_ampl)
    min_dur = .5 * np.min(flcd.flares.tstop-flcd.flares.tstart)
    print(min_dur)
    for i in range(3):
        flce, fake_flc = flcd.sample_flare_recovery(inject_before_detrending=True, mode="custom",
                                                    func=custom_detrending,
                                                  iterations=6, fakefreq=1e-3, ampl=[1e-2, max_ampl],
                                                  dur=[min_dur/6., 0.1/6.], save=True,
                                                  path="{}{}_{:012d}_s{:04d}.csv".format(i,5,
                                                                                         flc.targetid,
                                                                                         flc.campaign))

    flcc = flce.characterize_flares( ampl_bins=2, dur_bins=2)

/home/ekaterina/Documents/001_Science/Flares_in_Clusters_II/flaresinclustersii/lib/python3.6/site-packages/ipykernel_launcher.py:6: ResourceWarning: unclosed file <_io.FileIO name='/home/ekaterina/Documents/001_Science/TESS_UCDs/TESS_UCD_flares/custom_aperture/0ca_tess2019082014090-s0001-0000000029780677-0000-s_lc.fits' mode='rb' closefd=True>
  

De-trending mode custom does not exist. Pass "k2sc" (K2 LCs) or "savgol" (Kepler, TESS).
NoneType: None


0ca_tess2019082014090-s0001-0000000029780677-0000-s_lc.fits 



ValueError: 
De-trending mode custom does not exist. Pass "k2sc" (K2 LCs) or "savgol" (Kepler, TESS).

In [ ]:
flcc = flce.characterize_flares( ampl_bins=2, dur_bins=2)
flcc.flares.T

In [ ]:
# respath = f"{CWD}/flare_tables/2020_02_05_TESSUCDs_flares.csv"
# df = pd.read_csv(respath)
# df = df[df.ed_rec > 0] # keep only positive flare values
# df.to_csv(respath)

In [ ]:
# a = pd.DataFrame()
# for i, lf in df.groupby("TIC"):
#     a.loc[i,"nflares"] = int(lf.TIC.count())
#     a.loc[i,"nsectors"] = int(len(lf.Campaign.unique()))
# a["flarespersec"] = a.nflares/a.nsectors
# a.sort_values(by="flarespersec",ascending=False)


In [ ]:
# plt.figure(figsize=(16,5))
# markers =  ['o', 'v', '^', '<', '>', '8', 's', 'p', '*', 'h', 'H', 'D', 'd', 'P', 'X']
# for c, dfs in df.groupby("Campaign"):
#     plt.scatter(dfs.cstart, dfs.ampl_rec, label=f"{c}", marker=markers.pop())
# plt.legend(loc=(1.01,0))
# plt.title("cadence vs. flare amplitude by sector", fontsize=14);
# plt.yscale("log")
# plt.xlabel("cadence", fontsize=14)
# plt.ylabel("rel. amplitude", fontsize=14)
# plt.savefig("/home/ekaterina/Documents/001_Science/TESS_UCDs/TESS_UCD_flares/plots/TESS_UCD_flares_21_11_2019.png", dpi=300)

In [ ]:
# plt.figure(figsize=(16,5))
# markers =  ['o', 'v', '^', '<', '>', '8', 's', 'p', '*', 'h', 'H', 'D', 'd', 'P', 'X'] *100
# for c, dfs in df.groupby("TIC"):
#     plt.scatter(dfs.cstart, dfs.ampl_rec, label=f"TIC {c}", marker=markers.pop())
# #plt.legend()
# plt.title("cadence vs. flare amplitude by target");
# plt.yscale("log")
# plt.xlabel("cadence")
# plt.ylabel("rel. amplitude")


In [ ]:
df.shape

In [ ]:
# for i, path in enumerate(paths):
#     print(path, "\n")
#     path = f"{CWD}/custom_aperture/" + path
#     cluster = f"{CWD}/flare_tables/2019_11_13_TESSUCDs"

#     flc = read_custom_aperture_lc(path)
    
#     w = search_gaps_for_window_length(flc)
#     print(w)
#     flcd = custom_detrending(flc, d={flc.targetid:tuple(w)})

#     flcd = refine_detrended_flux_err(flcd)

#     flcd = flcd.find_flares()

#     write_flares_to_file(flcd, cluster)

#     print("\nFinished TIC {} ({})\n------------------------------\n".format(flc.targetid, flc.campaign))
